In [56]:
from newspaper import Article

import requests
import feedparser
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timezone


In [ ]:
# scrapping FranceTVInfo.fr

BASE_URL = "https://www.francetvinfo.fr"
CATEGORY = ["/societe","/monde","/politique"] 
MAX_PAGES = 5  # 50 max
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_article_urls(category, max_pages):
    urls = set()
    for page in range(2, max_pages + 2):  # 2.html → 51.html
        page_url = f"{BASE_URL}{category}/{page}.html"
        print(f"[{page}] Scraping: {page_url}")
        try:
            response = requests.get(page_url, headers=HEADERS)
            soup = BeautifulSoup(response.content, "html.parser")
            
            # chercher tous les articles de type 'card-article-list-l'
            articles = soup.find_all("article", class_="card-article-list-l")
            for article in articles:
                a_tag = article.find("a", class_="card-article-list-l__link")
                if a_tag and a_tag.get("href"):
                    href = a_tag["href"]
                    full_url = BASE_URL + href.strip()
                    urls.add(full_url)
        except Exception as e:
            print(f"Erreur sur la page {page}: {e}")
        time.sleep(1)
    return sorted(urls)

# ▶️ Récupération des URLs
for category in CATEGORY:
    article_urls = get_article_urls(category, MAX_PAGES)

# ▶️ Sauvegarde dans un fichier texte
with open("urls_franceinfo.txt", "w", encoding="utf-8") as f:
    for url in article_urls:
        f.write(url + "\n")

print(f"✅ {len(article_urls)} URLs enregistrées dans 'urls_franceinfo.txt'")

In [20]:
def extract_article(url):
    try:
        article = Article(url, language="fr")
        article.download()
        article.parse()
        return {
            "url": url,
            "title": article.title,
            "text": article.text,
            "publish_date": article.publish_date if article.publish_date else ""
        }
    except Exception as e:
        print(f"❌ Erreur avec {url} : {e}")
        return None

# ▶️ Lire les URLs depuis le fichier
with open("urls_franceinfo.txt", "r", encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

# ▶️ Extraction en boucle
articles_data = []
for i, url in enumerate(urls):
    print(f"[{i+1}/{len(urls)}] Extraction de : {url}")
    data = extract_article(url)
    if data and len(data["text"]) > 300:  # filtrer les articles trop courts
        articles_data.append(data)
    time.sleep(1)  # pause douce pour éviter les blocages

# ▶️ Enregistrement dans un CSV
df = pd.DataFrame(articles_data)
df.to_csv("articles_franceinfo_extraits.csv", index=False, encoding="utf-8")
print(f"✅ {len(df)} articles sauvegardés dans 'articles_franceinfo_extraits.csv'")

[1/145] Extraction de : https://www.francetvinfo.fr/economie/automobile/maires-en-colere-des-methodes-radicales-pour-faire-baisser-la-vitesse_7282128.html
[2/145] Extraction de : https://www.francetvinfo.fr/economie/budget/budget-f-bayrou-demande-un-effort-aux-francais_7277373.html
[3/145] Extraction de : https://www.francetvinfo.fr/economie/budget/budget-francois-bayrou-demande-aux-francais-de-faire-des-efforts_7276326.html
[4/145] Extraction de : https://www.francetvinfo.fr/economie/emploi/metiers/agriculture/pont-de-l-ascension-les-deputes-ne-siegeront-pas-les-30-et-31-mai-pour-examiner-le-texte-visant-a-lever-les-contraintes-dans-l-agriculture_7265805.html
[5/145] Extraction de : https://www.francetvinfo.fr/economie/emploi/metiers/agriculture/temoignages-c-est-une-vraie-coupe-budgetaire-les-professeurs-de-lycees-agricoles-protestent-contre-les-suppressions-de-postes-prevues_7263309.html
[6/145] Extraction de : https://www.francetvinfo.fr/economie/entreprises/qu-est-ce-que-le-devoir

In [46]:
url = "https://www.franceinfo.fr/monde/proche-orient/guerre/des-dockers-du-port-de-marseille-fos-refusent-de-charger-un-conteneur-avec-des-pieces-detachees-pour-fusils-mitrailleurs-a-destination-d-israel_7291656.html#xtor=RSS-3-[lestitres]"
article = Article(url, language="fr")
article.download()
article.parse()


In [43]:
print(article.title)
print(article.text)

Des dockers du port de Marseille-Fos refusent de charger un conteneur "avec des pièces détachées pour fusils-mitrailleurs" à destination d’Israël
Le syndicat général des dockers refuse de livrer ces armes, estimant qu'elles serviront "à l’armée israélienne pour continuer le massacre de la population palestinienne".

Un conteneur avec "des pièces détachées pour fusils mitrailleurs" a été mis de côté au port de Marseille-Fos par des dockers et ils "ne le chargeront pas sur le bateau à destination d’Haïfa", indique mercredi 4 juin le syndicat général de la CGT des ouvriers dockers et des personnels portuaires du Golfe de Fos dans un communiqué que franceinfo a consulté.

Le syndicat explique avoir "été alerté par plusieurs réseaux que 19 palettes de maillons d’eurolink devaient être embarquées par conteneur ce jeudi après-midi via le port de Marseille-Fos en direction du port d’Haïfa en Israël".

"Nous sommes pour la paix entre les peuples"

"Ces maillons d’eurolink sont des pièces détach

In [57]:

def fetch_articles_from_franceinfo(rss_url: str) -> list[dict]:
    feed = feedparser.parse(rss_url)
    print(feed)
    articles = []

    for entry in feed.entries:
        print(entry)
        article = {
            "title": entry.title,
            "summary": entry.get("summary", ""),
            "url": entry.link,
            "published_at": entry.get("published", datetime.now(timezone.utc).isoformat()),
            "fetched_at": datetime.now(timezone.utc).isoformat(),
            "source": "franceinfo",
            "language": "fr"
        }
        articles.append(article)

    return articles

url = 'https://www.franceinfo.fr/titres.rss'

print(fetch_articles_from_franceinfo(url))

{'bozo': False, 'entries': [{'title': '"Dati honoraire GDF Suez" : les troublants paiements reçus par Rachida Dati d\'un cabinet d\'avocats', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://www.franceinfo.fr/titres.rss', 'value': '"Dati honoraire GDF Suez" : les troublants paiements reçus par Rachida Dati d\'un cabinet d\'avocats'}, 'summary': '"Complément d\'enquête"\xa0et "Le Nouvel Obs"\xa0révèlent des documents exclusifs qui font apparaître que Rachida Dati a reçu, en 2010 et 2011, deux paiements de 149 500 euros chacun de la part d\'un cabinet d\'avocats auquel GDF Suez avait versé des sommes similaires. Alors députée au Parlement européen, elle prenait des positions jugées favorables au secteur gazier.', 'summary_detail': {'type': 'text/html', 'language': None, 'base': 'https://www.franceinfo.fr/titres.rss', 'value': '"Complément d\'enquête"\xa0et "Le Nouvel Obs"\xa0révèlent des documents exclusifs qui font apparaître que Rachida Dati a reçu, en 2010 et 

In [31]:
article

[]